# Carga de datos

Cargamos el dataset.

In [5]:
from sklearn.datasets import load_files
archivos = load_files('txt_sentoken')

# Clasificacion

Creamos el pipeline para clasificar las reviews de las peliculas. Elegimos Naive Bayes como clasificador.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

clasificador = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB(alpha=1.0))])
clasificador = clasificador.fit(archivos.data, archivos.target)

Prueba del clasificador.

In [9]:
predicciones = clasificador.predict(['Good movie', 'Bad terrible worst lamentable'])
print (predicciones)

[0 0]
